# Assignment 2: Korean to English Translation

- Sequence to Sequence 모델의 대표적인 한국어-영어 번역을 [Encoder-decoder](https://github.com/bentrevett/pytorch-seq2seq/blob/main/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb), [Attention]( https://github.com/bentrevett/pytorch-seq2seq/blob/main/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb), 그리고 [Transformers](https://github.com/bentrevett/pytorch-seq2seq/blob/main/legacy/6%20-%20Attention%20is%20All%20You%20Need.ipynb) 기반으로 구현
- Pytorch Seq to Seq 모델을 참고로 하여 한국어와 영어의 형태소분석되고 의존관계로 되어 있는 파일을 프로세싱하여 두 언어의 parallel 데이터 쌍으로 만들고 이를 학습하여 모델별로 Perplexity가 어떻게 달라지는지 살펴 보고, 가장 성능이 좋은 모델을 근간으로 해서 Inference로 한국어 문장을 입력하면 대응되는 영어 번역이 출력될 수 있도록 구현
- Transformer 기반은 이전 토치텍스트 버전으로 되어 있으니 이를 새로운 토치 텍스트 버전으로 바꾸어야 함
- 반드시 다음 세 모델에 대해서 PPL와 BLEU score가 다 체크되어야 함.  Encoder-Decoder, Attention, Transformers.
- 세 모델 중에 학습이 제대로 이루어지지 않는 경우, PPL이나 BLEU가 문제가 있는 경우 이를 Fix하려고 시도해 보라.
- **새로운 버전의 TorchText를 사용하여 코랩에서 실행가능하도록**
- 그룹을 허용. 그룹으로 할 경우 2명을 넘지 않아야 하며, 제출 파일에 참여자 이름과 역할을 반드시 명시할 것.
- Inference시에 unk인 단어를 로마자화해서 번역에 나타날 수 있도록 시도해 볼 것(참고할 수 있는 사이트 중 하나 https://github.com/osori/korean-romanizer)

## Data
- 첨부된 ko-en-en.parse.syn은 330,974 한국어 문장에 대응되는 영어문장이 품사와 구문분석이 되어 있는 파일이고 ko-en-ko.parse.syn은 이에 대응되는 한국어 문장이 형태소와 구문분석이 되어 있는 파일이다.

(ROOT (S (NP (NNP Flight) (NNP 007)) (VP (MD will) (VP (VB stay) (PP (IN on) (NP (NP (DT the) (NN ground)) (PP (IN for) (NP (CD one) (NN hour))))))) (. .)))


<id 1>
<sent 1>
1       2       NP      777/SN
2       6       NP_SBJ  항공편/NNG|은/JX
3       4       NP      1/SN|시간/NNG
4       6       NP_AJT  동안/NNG
5       6       NP_AJT  지상/NNG|에/JKB
6       7       VP      머물/VV|게/EC
7       0       VP      되/VV|ㅂ니다/EF|./SF
</sent>
</id>

- 이 두 파일을 프로세싱하여 한-영 병행 데이터로 만들고 이를 학습 및 테스트 데이터로 사용한다.
- Hint: 구조화된 데이터를 프로세싱하기 위해서는 nltk의 모듈을 사용할 수 있다.

- 한국어 형태소 분석된 단위를 어절별로 결합할 수 있고, 분석된 채로 그대로 사용할 수도 있다.
- 두 언어의 어순을 비슷하게 데이터를 만들어 학습할 수도 있고, 번역의 성능을 높이기 위해 다양한 형태로 재구조화 할 수 있다.

## Regarding torchtext version
- https://github.com/pytorch/text#installation
- torchtext 0.9.0 이전 버전은 torchtext.legacy로 변경됨; torchtext 0.12.0 버전 이후 legacy package가 제거되었음
- colab의 python, torch, torchtext 버전을 모두 맞춰주어야 함
- 계속 환경 및 버전을 맞춰보다가 거의 실패할 때쯤에 https://www.reddit.com/r/pytorch/comments/1eeochu/cant_import_torchtext/?rdt=64839 답변 참고해서 성공함
- 이번 과제에서 사용하는 건 0.15.2으로, new torchtext임


# 환경 갖추기

## 버전, 모듈 임포트 등

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install portalocker
import portalocker

In [ ]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
# not going to use torchdata, torchvision, torchaudio, so installation errors regarding those modules do not matter
!pip install torch==2.0.1
!pip install torchtext==0.15.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 56.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    F

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import torchtext
import tqdm

In [ ]:
print(torch.__version__)
print(torchtext.__version__)

2.0.1+cu117
0.15.2+cpu


In [ ]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# device 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터

## 파일 불러오기

In [ ]:
ko_path = "/content/drive/MyDrive/ko-en.ko.parse"
en_path = "/content/drive/MyDrive/ko-en.en.parse.syn"

In [ ]:
import pandas as pd

In [ ]:
ko_lines = ""
with open(ko_path, "r", encoding='utf-8') as ko_file:
    for line in ko_file.readlines():
        ko_lines += line

In [ ]:
with open(en_path, "r", encoding='utf-8') as en_file:
    en_lines = en_file.readlines()

## 전처리

In [ ]:
from nltk import Tree

In [ ]:
# nltk의 Tree 모듈을 사용하여 필요한 정보 추출, 각 문장에 <sos>, <eos> 넣어주기
en_text_list = []
for line in en_lines:
    sent = ''
    t = Tree.fromstring(line)
    for token in t.leaves():
      sent += token + ' '
    en_text_list.append(sent)

In [ ]:
en_text_list[:100]

['Flight 007 will stay on the ground for one hour . ',
 'Flight 017 will stay on the ground for three hours . ',
 "I need 1,000 dollars in traveler 's checks . ",
 'The official exchange rate is around 1,250 Won . ',
 'Please give me three hundred dollar bills and twenty dollar bills for the rest . ',
 'Can I have one hundred dollar bill and four fifty dollar bills ? ',
 'Do you have change for $ 100 ? ',
 "I 'd like to change 100 dollars . ",
 "I 'd like to change $ 100 . ",
 'Change 100 dollars . ',
 "I 'd like to change 100 dollars . ",
 'One hundred dollars . ',
 "I want four 100 's , two 20 's , five 10 's and ten 1 's . ",
 '6 ten dollar bills and 8 five dollar bills , please . ',
 'Could I have change for a one-hundred dollar bill ? ',
 "I 'd like to change one hundred . ",
 'I want four hundreds , three twenties , three tens , one five , and five ones . ',
 '100 miles is credited to your account . ',
 'Five tens , and ten twenties , please . ',
 'About 10 dollars . ',
 'I want 

In [ ]:
# 한 문장씩 나눈 리스트 만들기
sections = ko_lines.strip().split('\n\n')
ko_list = [section.splitlines() for section in sections]

ko_list[0]

['<id 1>',
 '<sent 1>',
 '1\t2\tNP\t777/SN',
 '2\t6\tNP_SBJ\t항공편/NNG|은/JX',
 '3\t4\tNP\t1/SN|시간/NNG',
 '4\t6\tNP_AJT\t동안/NNG',
 '5\t6\tNP_AJT\t지상/NNG|에/JKB',
 '6\t7\tVP\t머물/VV|게/EC',
 '7\t0\tVP\t되/VV|ㅂ니다/EF|./SF',
 '</sent>',
 '</id>']

In [ ]:
# 필요한 정보만 추출
import re

pattern = r"[가-힣ㄱ-ㅎ]+|[0-9]+(?=\/SN)"

for i in range(len(ko_list)):
    for j in range(len(ko_list[i])):
        ko_list[i][j] = re.findall(pattern, ko_list[i][j])

In [ ]:
ko_list[:50]

[[[],
  [],
  ['777'],
  ['항공편', '은'],
  ['1', '시간'],
  ['동안'],
  ['지상', '에'],
  ['머물', '게'],
  ['되', 'ㅂ니다'],
  [],
  []],
 [[],
  [],
  ['777'],
  ['항공편', '은'],
  ['3', '시간'],
  ['동안'],
  ['지상', '에'],
  ['있', '겠', '습니다'],
  [],
  []],
 [[], [], ['1', '000', '달러'], ['여행자', '수표', '가'], ['필요', '하', 'ㅂ니다'], [], []],
 [[], [], ['1', '250', '원', '이'], ['공식'], ['환율', '이', 'ㅂ니다'], [], []],
 [[],
  [],
  ['100', '달러'],
  ['3', '장', '과'],
  ['나머지', '는'],
  ['20', '달러', '권', '으로'],
  ['주', '시', 'ㅂ시오'],
  [],
  []],
 [[],
  [],
  ['100', '달러'],
  ['한'],
  ['장', '과'],
  ['50', '달러'],
  ['4', '장', '으로'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[], [], ['100', '달러', '를'], ['바꾸', '어'], ['주', '시', '겠', '어요'], [], []],
 [[], [], ['100', '달러', '만'], ['바꾸', '어'], ['주', '시', '어요'], [], []],
 [[],
  [],
  ['100', '달러', '만'],
  ['환전'],
  ['좀'],
  ['하', '아'],
  ['주', '시', '어요'],
  [],
  []],
 [[], [], ['100', '달러', '만'], ['환전', '하', '아'], ['주', '시', '어요'], [], []],
 [[], [], ['100', '달러', '만']

In [ ]:
# 문장으로 만들어서 리스트에 넣어주기, 각 문장에 <sos>, <eos> 넣어주기
ko_text_list = []

for i in range(len(ko_list)):
    sent = ''
    for j in range(len(ko_list[i])):
        if ko_list[i][j]:
            for token in ko_list[i][j]:
                sent += token + ' '
    ko_text_list.append(sent)

In [ ]:
ko_text_list[:100]

['777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 ',
 '777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 ',
 '1 000 달러 여행자 수표 가 필요 하 ㅂ니다 ',
 '1 250 원 이 공식 환율 이 ㅂ니다 ',
 '100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 ',
 '100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 ',
 '100 달러 를 바꾸 어 주 시 겠 어요 ',
 '100 달러 만 바꾸 어 주 시 어요 ',
 '100 달러 만 환전 좀 하 아 주 시 어요 ',
 '100 달러 만 환전 하 아 주 시 어요 ',
 '100 달러 만 환전 하 아 어 주 어 요 ',
 '100 달러 이 ㅂ니다 ',
 '100 달러 짜리 4 장 20 달러 짜리 2 장 10 달러 짜리 5 장 1 달러 짜리 10 장 으로 하 아 주 시 어요 ',
 '100 달러 짜리 지폐 6 개 하 고 5 달러 짜리 지폐 8 개 로 바꾸 어 주 시 어요 ',
 '100 달러 짜리 지폐 를 잔돈 으로 바꾸 ㄹ 수 있 겠 습니까 ',
 '100 불 을 바꾸 겠 습니다 ',
 '100 불 짜리 4 매 20 불 짜리 3 매 10 불 짜리 3 매 5 불 짜리 1 매 그리고 1 불 짜리 5 매 원하 ㅂ니다 ',
 '100 마일리지 가 적립 되 었 습니다 ',
 '10 달러 5 장 과 20 달러 10 장 으로 부탁 하 ㅂ니다 ',
 '10 달러 정도 이 에요 ',
 '10 달러 지폐 7 장 1 달러 지폐 30 장 주 시 어요 ',
 '10 달러 지폐 두 장 5 달러 짜리 두 장 주 어요 ',
 '10 달러 지폐 서른 장 과 잔돈 으로 부탁 하 아요 ',
 '10 달러 지폐 여덟 장하 고 5 달러 지폐 네 장 으로 부탁 하 아요 ',
 '10 달러 지폐 로 50 장 주 시 어요 ',
 '10 달러 지폐 를 잔돈 으로 바꾸 ㄹ 수 있 습니까 ',
 '10 달러 지폐 를 잔돈 으로 바꾸 ㄹ 려고요 ',
 '10 달러 는 현금 으로 하 고 나

## Dataset

In [ ]:
print(en_text_list[:10])
print(ko_text_list[:10])

['Flight 007 will stay on the ground for one hour . ', 'Flight 017 will stay on the ground for three hours . ', "I need 1,000 dollars in traveler 's checks . ", 'The official exchange rate is around 1,250 Won . ', 'Please give me three hundred dollar bills and twenty dollar bills for the rest . ', 'Can I have one hundred dollar bill and four fifty dollar bills ? ', 'Do you have change for $ 100 ? ', "I 'd like to change 100 dollars . ", "I 'd like to change $ 100 . ", 'Change 100 dollars . ']
['777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 ', '777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 ', '1 000 달러 여행자 수표 가 필요 하 ㅂ니다 ', '1 250 원 이 공식 환율 이 ㅂ니다 ', '100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 ', '100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 ', '100 달러 를 바꾸 어 주 시 겠 어요 ', '100 달러 만 바꾸 어 주 시 어요 ', '100 달러 만 환전 좀 하 아 주 시 어요 ', '100 달러 만 환전 하 아 주 시 어요 ']


In [ ]:
assert len(en_text_list) == len(ko_text_list), "Lists must be of equal length."

In [ ]:
sos_token = "<sos>"
eos_token = "<eos>"

def tokenize_with_special_tokens(text, type): # type: 0 for en, 1 for ko
    if type == 0:
        tokens = [token.lower() for token in text.split()]
    else:
        tokens = [token for token in text.split()]

    return [sos_token] + tokens + [eos_token]

In [ ]:
data = [
    {
        "en": en_text,
        "ko": ko_text,
        "en_tokens": tokenize_with_special_tokens(en_text, 0),
        "ko_tokens": tokenize_with_special_tokens(ko_text, 1),
    }
    for en_text, ko_text in zip(en_text_list, ko_text_list)
]

# Shuffle the data to randomize for train-test split.
random.shuffle(data)

# Define split ratios.
train_ratio, valid_ratio, test_ratio = 0.8, 0.1, 0.1
train_size = int(len(data) * train_ratio)
valid_size = int(len(data) * valid_ratio)

# Split the data.
train_data = data[:train_size]
valid_data = data[train_size:train_size + valid_size]
test_data = data[train_size + valid_size:]

train_data[0]

{'en': 'Does your watch keep good time ? ',
 'ko': '당신 시계 는 잘 맞 아요 ',
 'en_tokens': ['<sos>',
  'does',
  'your',
  'watch',
  'keep',
  'good',
  'time',
  '?',
  '<eos>'],
 'ko_tokens': ['<sos>', '당신', '시계', '는', '잘', '맞', '아요', '<eos>']}

## Vocabularies

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

# Collect all tokens from train_data for each language
def yield_tokens(data, key):
    for sample in data:
        yield sample[key]

# Define special tokens
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"
sos_token = "<sos>"
eos_token = "<eos>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

# Build vocabularies using the token iterator
en_vocab = build_vocab_from_iterator(
    yield_tokens(train_data, "en_tokens"),
    min_freq=min_freq,
    specials=special_tokens,
)

ko_vocab = build_vocab_from_iterator(
    yield_tokens(train_data, "ko_tokens"),
    min_freq=min_freq,
    specials=special_tokens,
)

In [ ]:
print(en_vocab.get_itos()[:10])
print(ko_vocab.get_itos()[:10])

['<unk>', '<pad>', '<sos>', '<eos>', '.', '?', 'i', 'you', 'the', 'to']
['<unk>', '<pad>', '<sos>', '<eos>', '이', '하', '시', '어요', '는', '가']


In [ ]:
len(en_vocab), len(ko_vocab)

(14212, 15986)

In [ ]:
assert en_vocab[unk_token] == ko_vocab[unk_token]
assert en_vocab[pad_token] == ko_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [ ]:
en_vocab.set_default_index(unk_index)
ko_vocab.set_default_index(unk_index)

In [ ]:
en_vocab.get_itos()[0]

'<unk>'

In [ ]:
tokens = ["i", "love", "watching", "crime", "shows"]

print(en_vocab.lookup_indices(tokens))
print(en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens)))

[6, 525, 1484, 5537, 1785]
['i', 'love', 'watching', 'crime', 'shows']


In [ ]:
def numericalize_example(example, en_vocab, ko_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    ko_ids = ko_vocab.lookup_indices(example["ko_tokens"])
    return {"en": example["en"], "ko": example["ko"], "en_tokens": example["en_tokens"], "ko_tokens": example["ko_tokens"], "en_ids": en_ids, "ko_ids": ko_ids}

In [ ]:
fn_kwargs = {"en_vocab": en_vocab, "ko_vocab": ko_vocab}

train_data = [numericalize_example(example, **fn_kwargs) for example in train_data]
valid_data = [numericalize_example(example, **fn_kwargs) for example in valid_data]
test_data = [numericalize_example(example, **fn_kwargs) for example in test_data]

In [ ]:
train_data[0]

{'en': 'Does your watch keep good time ? ',
 'ko': '당신 시계 는 잘 맞 아요 ',
 'en_tokens': ['<sos>',
  'does',
  'your',
  'watch',
  'keep',
  'good',
  'time',
  '?',
  '<eos>'],
 'ko_tokens': ['<sos>', '당신', '시계', '는', '잘', '맞', '아요', '<eos>'],
 'en_ids': [2, 49, 30, 370, 192, 80, 51, 5, 3],
 'ko_ids': [2, 79, 640, 8, 86, 167, 46, 3]}

In [ ]:
en_vocab.lookup_tokens(train_data[0]["en_ids"])

['<sos>', 'does', 'your', 'watch', 'keep', 'good', 'time', '?', '<eos>']

In [ ]:
data_type = "torch"
format_columns = ["en_ids", "ko_ids"]

for example in train_data:
    for column in format_columns:
        example[column] = torch.tensor(example[column], dtype=torch.long)

for example in valid_data:
    for column in format_columns:
        example[column] = torch.tensor(example[column], dtype=torch.long)

for example in test_data:
    for column in format_columns:
        example[column] = torch.tensor(example[column], dtype=torch.long)

print(train_data[0])
print(valid_data[0])
print(test_data[0])

{'en': 'Does your watch keep good time ? ', 'ko': '당신 시계 는 잘 맞 아요 ', 'en_tokens': ['<sos>', 'does', 'your', 'watch', 'keep', 'good', 'time', '?', '<eos>'], 'ko_tokens': ['<sos>', '당신', '시계', '는', '잘', '맞', '아요', '<eos>'], 'en_ids': tensor([  2,  49,  30, 370, 192,  80,  51,   5,   3]), 'ko_ids': tensor([  2,  79, 640,   8,  86, 167,  46,   3])}
{'en': 'Would you please tell me how to open this gateway ? ', 'ko': '이 출입구 열 는 법 을 가르치 어 주 시 겠 습니까 ', 'en_tokens': ['<sos>', 'would', 'you', 'please', 'tell', 'me', 'how', 'to', 'open', 'this', 'gateway', '?', '<eos>'], 'ko_tokens': ['<sos>', '이', '출입구', '열', '는', '법', '을', '가르치', '어', '주', '시', '겠', '습니까', '<eos>'], 'en_ids': tensor([   2,   46,    7,   23,   87,   22,   18,    9,  212,   20, 9497,    5,
           3]), 'ko_ids': tensor([   2,    4, 4123,  367,    8,  895,   12,  146,   17,   14,    6,   21,
          26,    3])}
{'en': "I 'm looking for marine products . ", 'ko': '수산물 을 찾 고 있 는데요 ', 'en_tokens': ['<sos>', 'i', "'m", 'looking'

In [ ]:
type(train_data[0]["en_ids"])

torch.Tensor

## Data Loaders

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_ko_ids = [example["ko_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_ko_ids = nn.utils.rnn.pad_sequence(batch_ko_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "ko_ids": batch_ko_ids,
        }
        return batch

    return collate_fn

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [ ]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [ ]:
batch = next(iter(train_data_loader))

print(batch['en_ids'].shape)
print(batch['ko_ids'].shape)

torch.Size([23, 128])
torch.Size([27, 128])


# Encoder-decoder

## Build the Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

## Train the Model

In [ ]:
input_dim = len(ko_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(15986, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(14212, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=14212, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 22,377,860 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

### Training Loop

In [ ]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["ko_ids"].to(device)
        trg = batch["en_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

### Evaluation Loop

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["ko_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

### Model Training

In [ ]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "encoder-decoder-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 10%|█         | 1/10 [04:16<38:24, 256.03s/it]

	Train Loss:   4.324 | Train PPL:  75.502
	Valid Loss:   4.344 | Valid PPL:  76.984


 20%|██        | 2/10 [08:31<34:06, 255.75s/it]

	Train Loss:   3.188 | Train PPL:  24.234
	Valid Loss:   3.462 | Valid PPL:  31.884


 30%|███       | 3/10 [12:47<29:50, 255.76s/it]

...

	Train Loss:   1.728 | Train PPL:   5.628
	Valid Loss:   2.595 | Valid PPL:  13.402


 90%|█████████ | 9/10 [38:22<04:15, 255.89s/it]

	Train Loss:   1.656 | Train PPL:   5.238
	Valid Loss:   2.543 | Valid PPL:  12.720


100%|██████████| 10/10 [42:38<00:00, 255.83s/it]

	Train Loss:   1.583 | Train PPL:   4.870
	Valid Loss:   2.514 | Valid PPL:  12.359


## Evaluate the Model

In [ ]:
model.load_state_dict(torch.load("encoder-decoder-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 2.508 | Test PPL:  12.286 |


## Inference

In [ ]:
import torch

def translate_sentence(
    sentence,
    model,
    en_vocab,
    ko_vocab,
    sos_token="<sos>",
    eos_token="<eos>",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    max_output_length=25,
    lower=True
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = sentence.split()
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = ko_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)

    return tokens

In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']

In [ ]:
translated_sentences = []
for sentence in sen_list:
    translation = translate_sentence(
        sentence=sentence,
        model=model,
        en_vocab=en_vocab,
        ko_vocab=ko_vocab,
        sos_token="<sos>",
        eos_token="<eos>",
        device=device,
        max_output_length=25
    )
    translated_sentences.append(translation)

for original, translation in zip(sen_list, translated_sentences):
    print(f"Original: {original}")
    filtered_translation = [token for token in translation if token not in ["<sos>", "<eos>"]]
    print(f"Translation: {' '.join(filtered_translation)}\n")

Original: 모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .
Translation: all liquids , gels , gels , gels , zip-top , zip-top , zip-top , zip-top , zip-top , one-quart plastic bag .

Original: 미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?
Translation: i 'm sorry , but i 'm to the the <unk> for the <unk> . could you please give me to the <unk> on the

Original: 은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요
Translation: the 's too much . i you give me a parking lot of the it 's not working .

Original: 아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?
Translation: i think we have lost the computer , so we can have to have lost our baggage . could you send our name ?

Original: 부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .
Translation: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> .

Original: 변기 가 막히 었 습니다 .
Translation: the toilet does n't flush .

Original: 그 바지 좀 보이 어 주 시 ㅂ시오 . 이

## Bleu Score

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(reference, prediction, weights=[1, 0, 0, 0]):
    score = corpus_bleu(reference, prediction, weights=weights)
    return score

In [ ]:
translations = [
    translate_sentence(
        example["ko"],
        model,
        en_vocab,
        ko_vocab,
        sos_token,
        eos_token,
        device,
        max_output_length=25,
        lower=True,
    )
    for example in tqdm.tqdm(test_data)
]

100%|██████████| 33098/33098 [02:52<00:00, 192.25it/s]


In [ ]:
reference = [example["en"] for example in test_data]
prediction = [" ".join([token for token in translation if token not in ["<sos>", "<eos>"]]) for translation in translations]

print(reference[:5])
print(prediction[:5])

["I 'm looking for marine products . ", 'Can I get off at the Seouryeoksabangmulgwan ? ', 'I just got here this morning . ', 'Can I help you ? ', "There 's a police station across the street . I 'm sure they can help you . "]
["i 'm looking for a <unk> .", 'can i get off at the seouryeoksabangmulgwan ?', "i 'm here here this morning .", 'may i ask you you ?', "there 's a police stand across the street . you can see the way ."]


In [ ]:
bleu_score = calculate_bleu(reference, prediction)
print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 39.68


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

# Attention

## Build the Model

In [ ]:
class Encoder(nn.Module):
    def __init__(
        self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout
    ):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, bidirectional=True)
        self.fc = nn.Linear(encoder_hidden_dim * 2, decoder_hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        # hidden [-2, :, : ] is the last of the forwards RNN
        # hidden [-1, :, : ] is the last of the backwards RNN
        # initial decoder hidden is final hidden state of the forwards and backwards
        # encoder RNNs fed through a linear layer
        hidden = torch.tanh(
            self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        )
        # outputs = [src length, batch size, encoder hidden dim * 2]
        # hidden = [batch size, decoder hidden dim]
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        self.attn_fc = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim, decoder_hidden_dim
        )
        self.v_fc = nn.Linear(decoder_hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden = [batch size, decoder hidden dim]
        # encoder_outputs = [src length, batch size, encoder hidden dim * 2]
        batch_size = encoder_outputs.shape[1]
        src_length = encoder_outputs.shape[0]
        # repeat decoder hidden state src_length times
        hidden = hidden.unsqueeze(1).repeat(1, src_length, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # hidden = [batch size, src length, decoder hidden dim]
        # encoder_outputs = [batch size, src length, encoder hidden dim * 2]
        energy = torch.tanh(self.attn_fc(torch.cat((hidden, encoder_outputs), dim=2)))
        # energy = [batch size, src length, decoder hidden dim]
        attention = self.v_fc(energy).squeeze(2)
        # attention = [batch size, src length]
        return torch.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim,
        embedding_dim,
        encoder_hidden_dim,
        decoder_hidden_dim,
        dropout,
        attention,
    ):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.GRU((encoder_hidden_dim * 2) + embedding_dim, decoder_hidden_dim)
        self.fc_out = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim + embedding_dim, output_dim
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        # input = [batch size]
        # hidden = [batch size, decoder hidden dim]
        # encoder_outputs = [src length, batch size, encoder hidden dim * 2]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        a = self.attention(hidden, encoder_outputs)
        # a = [batch size, src length]
        a = a.unsqueeze(1)
        # a = [batch size, 1, src length]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # encoder_outputs = [batch size, src length, encoder hidden dim * 2]
        weighted = torch.bmm(a, encoder_outputs)
        # weighted = [batch size, 1, encoder hidden dim * 2]
        weighted = weighted.permute(1, 0, 2)
        # weighted = [1, batch size, encoder hidden dim * 2]
        rnn_input = torch.cat((embedded, weighted), dim=2)
        # rnn_input = [1, batch size, (encoder hidden dim * 2) + embedding dim]
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        # output = [seq length, batch size, decoder hid dim * n directions]
        # hidden = [n layers * n directions, batch size, decoder hid dim]
        # seq len, n layers and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, decoder hidden dim]
        # hidden = [1, batch size, decoder hidden dim]
        # this also means that output == hidden
        assert (output == hidden).all()
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        # prediction = [batch size, output dim]
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        batch_size = src.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # encoder_outputs is all hidden states of the input sequence, back and forwards
        # hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
        # outputs = [src length, batch size, encoder hidden dim * 2]
        # hidden = [batch size, decoder hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden state and all encoder hidden states
            # receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, decoder hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

## Train the Model

In [ ]:
input_dim = len(ko_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_hidden_dim = 512
decoder_hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

attention = Attention(encoder_hidden_dim, decoder_hidden_dim)

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    decoder_dropout,
    attention,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if "weight" in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(15986, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_fc): Linear(in_features=1536, out_features=512, bias=True)
      (v_fc): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(14212, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=14212, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 39,646,084 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

### Training Loop

In [ ]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["ko_ids"].to(device)
        trg = batch["en_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

### Evaluation Loop

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["ko_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

### Model Training

In [ ]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "attention-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 10%|█         | 1/10 [06:08<55:16, 368.47s/it]

	Train Loss:   3.322 | Train PPL:  27.726
	Valid Loss:   2.931 | Valid PPL:  18.750


 20%|██        | 2/10 [12:18<49:13, 369.13s/it]

	Train Loss:   2.068 | Train PPL:   7.910
	Valid Loss:   2.535 | Valid PPL:  12.611


 30%|███       | 3/10 [18:26<43:01, 368.82s/it]

...

	Train Loss:   1.181 | Train PPL:   3.258
	Valid Loss:   2.273 | Valid PPL:   9.707


 90%|█████████ | 9/10 [55:15<06:08, 368.57s/it]

	Train Loss:   1.143 | Train PPL:   3.135
	Valid Loss:   2.259 | Valid PPL:   9.574


100%|██████████| 10/10 [1:01:26<00:00, 368.64s/it]

	Train Loss:   1.104 | Train PPL:   3.017
	Valid Loss:   2.265 | Valid PPL:   9.632


## Evaluate the Model

In [ ]:
model.load_state_dict(torch.load("attention-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 2.240 | Test PPL:   9.391 |


## Inference

In [ ]:
import torch

def translate_sentence(
    sentence,
    model,
    en_vocab,
    ko_vocab,
    sos_token="<sos>",
    eos_token="<eos>",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    max_output_length=25,
    lower=True
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = sentence.split()
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = ko_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)

        encoder_outputs, hidden = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        attentions = []
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, attention = model.decoder(
                inputs_tensor, hidden, encoder_outputs
            )
            attentions.append(attention.squeeze(0).cpu())
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break

        tokens = en_vocab.lookup_tokens(inputs)

    attentions = torch.stack(attentions)
    return tokens, attentions

In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']

In [ ]:
translated_sentences = []
for sentence in sen_list:
    translation, attentions = translate_sentence(
        sentence=sentence,
        model=model,
        en_vocab=en_vocab,
        ko_vocab=ko_vocab,
        sos_token="<sos>",
        eos_token="<eos>",
        device=device,
        max_output_length=25
    )
    translated_sentences.append(translation)

for original, translation in zip(sen_list, translated_sentences):
    print(f"Original: {original}")
    filtered_translation = [token for token in translation if token not in ["<sos>", "<eos>"]]
    print(f"Translation: {' '.join(filtered_translation)}\n")

Original: 모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .
Translation: all liquids , gels and aerosols must be placed in a single , and aerosols in a single plastic bag , and a plastic bag

Original: 미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?
Translation: i 'm sorry , but i have to move to <unk> <unk> . could i change my ticket to <unk> <unk> .

Original: 은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요
Translation: that bank is too bad . you you need to buy some money cash or money ?

Original: 아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?
Translation: i may have lost loss , may i have to ship the baggage report . then . do you you to come to the office

Original: 부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .
Translation: the <unk> <unk> the <unk> <unk> <unk> <unk> <unk> and <unk> <unk> <unk> <unk> <unk> <unk> <unk> .

Original: 변기 가 막히 었 습니다 .
Translation: the toilet is clogged .

Original: 그 바지 좀 보이 어 주 시 ㅂ시오

## Bleu Score

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(reference, prediction, weights=[1, 0, 0, 0]):
    score = corpus_bleu(reference, prediction, weights=weights)
    return score

In [ ]:
translations = [
    translate_sentence(
        example["ko"],
        model,
        en_vocab,
        ko_vocab,
        sos_token,
        eos_token,
        device,
        max_output_length=25,
        lower=True,
    )
    for example in tqdm.tqdm(test_data)
]

100%|██████████| 33098/33098 [05:57<00:00, 92.67it/s]


In [ ]:
print(translations[:5])

[(['<sos>', 'i', "'m", 'looking', 'for', 'a', '<unk>', 'rice', 'cooker', '.', '<eos>'], tensor([[1.5945e-02, 1.4400e-01, 3.2376e-02, 2.1313e-02, 1.4572e-01, 5.4628e-01,
         2.8635e-03, 9.1506e-02],
        [1.8612e-02, 1.9571e-01, 1.2708e-01, 1.8384e-02, 4.8681e-02, 5.0221e-01,
         1.9165e-03, 8.7399e-02],
        [4.9689e-03, 1.2740e-01, 1.0835e-01, 7.3191e-03, 7.7570e-02, 5.3438e-01,
         7.1126e-04, 1.3930e-01],
        [2.4011e-02, 5.6920e-01, 1.3364e-01, 5.8792e-03, 3.2484e-02, 1.6364e-01,
         5.5477e-04, 7.0583e-02],
        [1.5532e-02, 7.9818e-01, 7.9315e-02, 3.9566e-03, 1.2931e-02, 3.5891e-02,
         2.6419e-04, 5.3931e-02],
        [2.0228e-02, 8.0961e-01, 4.9750e-02, 3.0977e-03, 2.1421e-02, 3.6902e-02,
         9.4605e-04, 5.8043e-02],
        [2.2163e-02, 6.6409e-01, 1.6977e-01, 3.6386e-03, 2.0855e-02, 3.9728e-02,
         1.4730e-03, 7.8289e-02],
        [1.6622e-02, 7.8706e-01, 6.5737e-02, 2.5083e-03, 1.3994e-02, 4.9837e-02,
         1.9007e-03, 6.234

In [ ]:
reference = [example["en"] for example in test_data]
prediction = [
    " ".join([token for token in translation[0] if token not in ["<sos>", "<eos>"]])
    for translation in translations
]

print(reference[:5])
print(prediction[:5])

["I 'm looking for marine products . ", 'Can I get off at the Seouryeoksabangmulgwan ? ', 'I just got here this morning . ', 'Can I help you ? ', "There 's a police station across the street . I 'm sure they can help you . "]
["i 'm looking for a <unk> rice cooker .", 'can i get off at the seouryeoksabangmulgwan ?', 'i arrived here this morning .', 'would you tell me to call ?', "there 's a police station across the street . i can sure they can help you ."]


In [ ]:
bleu_score = calculate_bleu(reference, prediction)
print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 37.83


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

# Transformers

## Build the Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim,
                 hid_dim,
                 n_layers,
                 n_heads,
                 pf_dim,
                 dropout,
                 device,
                 max_length=100):
        super().__init__()

        self.device = device
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        self.layers = nn.ModuleList([
            EncoderLayer(hid_dim, n_heads, pf_dim, dropout, device)
            for _ in range(n_layers)
        ])
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

    def forward(self, src, src_mask):
        # src = [src len, batch size]
        # src_mask = [batch size, 1, 1, src len]

        batch_size = src.shape[1]
        src_len = src.shape[0]

        pos = torch.arange(0, src_len).unsqueeze(1).repeat(1, batch_size).to(self.device)
        # pos = [src len, batch size]

        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        # src = [src len, batch size, hid dim]

        for layer in self.layers:
            src = layer(src, src_mask)
        # src = [src len, batch size, hid dim]

        return src

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self,
                 hid_dim,
                 n_heads,
                 pf_dim,
                 dropout,
                 device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim,
                                                                     pf_dim,
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask):
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len]

        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)

        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        #src = [batch size, src len, hid dim]

        #positionwise feedforward
        _src = self.positionwise_feedforward(src)

        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        #src = [batch size, src len, hid dim]

        return src

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()

        assert hid_dim % n_heads == 0

        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads

        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)

        self.fc_o = nn.Linear(hid_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask=None):
        # query, key, value = [seq len, batch size, hid dim]

        batch_size = query.shape[1]

        # Fully connected layers
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        # Q, K, V = [seq len, batch size, hid dim]

        # Reshape to [seq len, batch size, n heads, head dim] and permute
        Q = Q.view(-1, batch_size, self.n_heads, self.head_dim).permute(1, 2, 0, 3)
        K = K.view(-1, batch_size, self.n_heads, self.head_dim).permute(1, 2, 0, 3)
        V = V.view(-1, batch_size, self.n_heads, self.head_dim).permute(1, 2, 0, 3)
        # Q, K, V = [batch size, n heads, seq len, head dim]

        # Scaled dot-product attention
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        # energy = [batch size, n heads, seq len, seq len]

        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = torch.softmax(energy, dim=-1)
        # attention = [batch size, n heads, seq len, seq len]

        x = torch.matmul(self.dropout(attention), V)
        # x = [batch size, n heads, seq len, head dim]

        x = x.permute(2, 0, 1, 3).contiguous()
        # x = [seq len, batch size, n heads, head dim]

        x = x.view(-1, batch_size, self.hid_dim)
        # x = [seq len, batch size, hid dim]

        x = self.fc_o(x)
        # x = [seq len, batch size, hid dim]

        return x, attention

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()

        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #x = [batch size, seq len, hid dim]

        x = self.dropout(torch.relu(self.fc_1(x)))
        #x = [batch size, seq len, pf dim]

        x = self.fc_2(x)
        #x = [batch size, seq len, hid dim]

        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim,
                 hid_dim,
                 n_layers,
                 n_heads,
                 pf_dim,
                 dropout,
                 device,
                 max_length=100):
        super().__init__()

        self.device = device
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        self.layers = nn.ModuleList([
            DecoderLayer(hid_dim, n_heads, pf_dim, dropout, device)
            for _ in range(n_layers)
        ])
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        # trg = [trg len, batch size]
        # enc_src = [src len, batch size, hid dim]
        # trg_mask = [batch size, 1, trg len, trg len]
        # src_mask = [batch size, 1, 1, src len]

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]

        pos = torch.arange(0, trg_len).unsqueeze(1).repeat(1, batch_size).to(self.device)
        # pos = [trg len, batch size]

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
        # trg = [trg len, batch size, hid dim]

        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        # trg = [trg len, batch size, hid dim]
        # attention = [batch size, n heads, trg len, src len]

        output = self.fc_out(trg)
        # output = [trg len, batch size, output dim]

        return output, attention

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self,
                 hid_dim,
                 n_heads,
                 pf_dim,
                 dropout,
                 device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim,
                                                                     pf_dim,
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, enc_src, trg_mask, src_mask):

        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]

        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
        #trg = [batch size, trg len, hid dim]

        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
        #trg = [batch size, trg len, hid dim]

        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]

        return trg, attention

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        # src = [src len, batch size]
        src_mask = (src != self.src_pad_idx).permute(1, 0).unsqueeze(1).unsqueeze(2)
        # src_mask = [batch size, 1, 1, src len]
        return src_mask

    def make_trg_mask(self, trg):
        # trg = [trg len, batch size]
        trg_pad_mask = (trg != self.trg_pad_idx).permute(1, 0).unsqueeze(1).unsqueeze(2)
        # trg_pad_mask = [batch size, 1, 1, trg len]

        trg_len = trg.shape[0]
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=self.device)).bool()
        # trg_sub_mask = [trg len, trg len]

        trg_sub_mask = trg_sub_mask.unsqueeze(0).unsqueeze(0)
        # trg_sub_mask = [1, 1, trg len, trg len]

        trg_mask = trg_pad_mask & trg_sub_mask
        # trg_mask = [batch size, 1, trg len, trg len]

        return trg_mask

    def forward(self, src, trg):
        # src = [src len, batch size]
        # trg = [trg len, batch size]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        # src_mask = [batch size, 1, 1, src len]
        # trg_mask = [batch size, 1, trg len, trg len]

        enc_src = self.encoder(src, src_mask)
        # enc_src = [src len, batch size, hid dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        # output = [trg len, batch size, output dim]
        # attention = [batch size, n heads, trg len, src len]

        return output, attention

## Train the Model

In [ ]:
INPUT_DIM = len(ko_vocab)
OUTPUT_DIM = len(en_vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM,
              HID_DIM,
              ENC_LAYERS,
              ENC_HEADS,
              ENC_PF_DIM,
              ENC_DROPOUT,
              device)

dec = Decoder(OUTPUT_DIM,
              HID_DIM,
              DEC_LAYERS,
              DEC_HEADS,
              DEC_PF_DIM,
              DEC_DROPOUT,
              device)

model = Seq2Seq(enc, dec, pad_index, pad_index, device).to(device)

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Seq2Seq(
  (encoder): Encoder(
    (tok_embedding): Embedding(15986, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0-2): 3 x EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 15,388,036 trainable parameters


In [ ]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = pad_index)

### Training Loop

In [ ]:
def train_fn(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch["ko_ids"].to(device)  # [src len, batch size]
        trg = batch["en_ids"].to(device)  # [trg len, batch size]

        optimizer.zero_grad()

        output, _ = model(src, trg[:-1, :])  # trg의 마지막 단어 제외
        # output = [trg len - 1, batch size, output dim]
        # trg = [trg len, batch size]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        trg = trg[1:].contiguous().view(-1)  # trg의 첫 단어 제외
        # output = [(trg len - 1) * batch size, output dim]
        # trg = [(trg len - 1) * batch size]

        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

### Evaluation Loop

In [ ]:
def evaluate_fn(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch["ko_ids"].to(device)  # [src len, batch size]
            trg = batch["en_ids"].to(device)  # [trg len, batch size]

            output, _ = model(src, trg[:-1, :])  # trg의 마지막 단어 제외
            # output = [trg len - 1, batch size, output dim]
            # trg = [trg len, batch size]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            trg = trg[1:].contiguous().view(-1)  # trg의 첫 단어 제외
            # output = [(trg len - 1) * batch size, output dim]
            # trg = [(trg len - 1) * batch size]

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

### Model Training

In [ ]:
n_epochs = 10
clip = 1.0

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "transformers-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 10%|█         | 1/10 [02:47<25:03, 167.01s/it]

	Train Loss:   1.440 | Train PPL:   4.223
	Valid Loss:   1.315 | Valid PPL:   3.724


 20%|██        | 2/10 [05:30<21:59, 164.91s/it]

	Train Loss:   1.189 | Train PPL:   3.284
	Valid Loss:   1.185 | Valid PPL:   3.270


 30%|███       | 3/10 [08:09<18:55, 162.19s/it]

...

	Train Loss:   0.724 | Train PPL:   2.063
	Valid Loss:   0.975 | Valid PPL:   2.651


 90%|█████████ | 9/10 [24:00<02:38, 158.70s/it]

	Train Loss:   0.691 | Train PPL:   1.996
	Valid Loss:   0.964 | Valid PPL:   2.623


100%|██████████| 10/10 [26:39<00:00, 159.91s/it]

	Train Loss:   0.663 | Train PPL:   1.941
	Valid Loss:   0.950 | Valid PPL:   2.587


## Evaluate the Model

In [ ]:
model.load_state_dict(torch.load("transformers-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 0.951 | Test PPL:   2.589 |


## Inference

In [ ]:
def translate_sentence(
    sentence,
    model,
    en_vocab,
    ko_vocab,
    sos_token="<sos>",
    eos_token="<eos>",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    max_output_length=25,
    lower=True
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = sentence.split()
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        src_indices = ko_vocab.lookup_indices(tokens)
        src_tensor = torch.LongTensor(src_indices).unsqueeze(1).to(device)

        src_mask = model.make_src_mask(src_tensor)
        enc_src = model.encoder(src_tensor, src_mask)

        trg_indices = [en_vocab[sos_token]]
        attentions = []

        for _ in range(max_output_length):
            trg_tensor = torch.LongTensor(trg_indices).unsqueeze(1).to(device)
            trg_mask = model.make_trg_mask(trg_tensor)

            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
            attentions.append(attention.cpu())

            predicted_token = output.argmax(-1)[-1].item()
            trg_indices.append(predicted_token)

            if predicted_token == en_vocab[eos_token]:
                break

        translated_tokens = en_vocab.lookup_tokens(trg_indices)

    return translated_tokens, attentions

In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']

In [ ]:
translated_sentences = []
for sentence in sen_list:
    translation, attentions = translate_sentence(
        sentence=sentence,
        model=model,
        en_vocab=en_vocab,
        ko_vocab=ko_vocab,
        sos_token="<sos>",
        eos_token="<eos>",
        device=device,
        max_output_length=25
    )
    translated_sentences.append(translation)

for original, translation in zip(sen_list, translated_sentences):
    print(f"Original: {original}")
    filtered_translation = [token for token in translation if token not in ["<sos>", "<eos>"]]
    print(f"Translation: {' '.join(filtered_translation)}\n")

Original: 모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .
Translation: all liquids , gels and aerosols , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , and may

Original: 미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?
Translation: i 'm sorry , miss. . <unk> <unk> the <unk> of communication companies in gwanghwamun . could you change the <unk> ?

Original: 은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요
Translation: the bank is too far . i need to steal someone else <unk> .

Original: 아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?
Translation: we 've lost the error of our loss , we have to fill out the office <unk> . would you like to join us ?

Original: 부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .
Translation: busan is going through the street <unk> of <unk> .

Original: 변기 가 막히 었 습니다 .
Translation: the toilet does n't flush .

Original: 그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?
Translatio

## Bleu Score

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(reference, prediction, weights=[1, 0, 0, 0]):
    score = corpus_bleu(reference, prediction, weights=weights)
    return score

In [ ]:
translations = [
    translate_sentence(
        example["ko"],
        model,
        en_vocab,
        ko_vocab,
        sos_token,
        eos_token,
        device,
        max_output_length=25,
        lower=True,
    )
    for example in tqdm.tqdm(test_data)
]

100%|██████████| 33098/33098 [24:37<00:00, 22.40it/s]


In [ ]:
print(translations[:5])

[(['<sos>', 'i', "'m", 'looking', 'for', 'a', 'spoon', 'stand', '.', '<eos>'], [tensor([[[[0.1809, 0.2804, 0.1938, 0.0308, 0.0453, 0.0505, 0.0336, 0.1849]],

         [[0.0350, 0.0707, 0.1608, 0.0812, 0.1926, 0.1594, 0.2653, 0.0350]],

         [[0.0911, 0.0597, 0.1563, 0.0789, 0.1440, 0.1262, 0.2565, 0.0873]],

         [[0.0934, 0.0896, 0.2173, 0.0895, 0.1479, 0.0563, 0.2165, 0.0895]],

         [[0.1822, 0.1334, 0.1522, 0.0392, 0.0903, 0.0649, 0.1659, 0.1719]],

         [[0.1431, 0.1238, 0.1823, 0.0603, 0.1085, 0.1035, 0.1421, 0.1364]],

         [[0.0324, 0.0475, 0.2220, 0.1067, 0.1826, 0.0820, 0.2950, 0.0319]],

         [[0.0310, 0.0379, 0.1628, 0.0537, 0.2239, 0.1650, 0.2948, 0.0308]]]]), tensor([[[[0.1809, 0.2804, 0.1938, 0.0308, 0.0453, 0.0505, 0.0336, 0.1849],
          [0.0075, 0.0122, 0.1085, 0.2855, 0.1220, 0.2123, 0.2447, 0.0073]],

         [[0.0350, 0.0707, 0.1608, 0.0812, 0.1926, 0.1594, 0.2653, 0.0350],
          [0.0073, 0.0050, 0.1398, 0.4022, 0.1754, 0.0927, 0.170

In [ ]:
reference = [example["en"] for example in test_data]
prediction = [
    " ".join([token for token in translation[0] if token not in ["<sos>", "<eos>"]])
    for translation in translations
]

print(reference[:5])
print(prediction[:5])

["I 'm looking for marine products . ", 'Can I get off at the Seouryeoksabangmulgwan ? ', 'I just got here this morning . ', 'Can I help you ? ', "There 's a police station across the street . I 'm sure they can help you . "]
["i 'm looking for a spoon stand .", 'can i get off at the seouryeoksabangmulgwan ?', 'i got here this morning .', 'may i help you ?', "there 's a police station across the street . i 'm sure we 'll help you ."]


In [ ]:
bleu_score = calculate_bleu(reference, prediction)
print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 39.55


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_